## ЛУ 4: Манипулация на Качетвени данни

### Теория

### Работа с качествени данни в pandas

Библиотеката pandas предоставя и множество методи за работа с качествени данни, които най-често идват под формата на низове (strings).

In [1]:
import pandas as pd

dataframe_length = 100
dict_for_pd_df = {
    'string_column': ['string_value_'+str(i) for i in range(dataframe_length)],
    'binary_column': [True if (i+1) % 2==0 else False for i in range(dataframe_length)], #Ако остатъкът от делене на 2 е 0, тогава задаваме вярна стойност: тази колонка проверява дали имаме четни числа или не (добавяме 1, за да отговаря на int_column).
    'list_column': [[i, i*2, i*3] for i in range(dataframe_length)],
    'date_column': ['01/01/' + str(1900+i) for i in range(dataframe_length)],
    'three_categories': ['A']*(dataframe_length//3) + ['B']*(dataframe_length//3) + ['C']*(dataframe_length//3 + dataframe_length%3),
    'dollar_values': ['$'+str(i*100) for i in range(dataframe_length)]
}
my_first_df = pd.DataFrame(dict_for_pd_df)

Основният начин да разгледаме качествена променлива е чрез метода value_counts, който показва колко стойности имаме за дадена променлива.

In [2]:
my_first_df.three_categories.value_counts()

C    34
A    33
B    33
Name: three_categories, dtype: int64

In [3]:
my_first_df.binary_column.value_counts()

False    50
True     50
Name: binary_column, dtype: int64

Също така, можем да искаме да извадим списък с различните уникални стойности чрез unique или пък да извикаме само техния брой, ако колонката има твърде много уникални стойности. 

In [4]:
my_first_df.three_categories.unique()

array(['A', 'B', 'C'], dtype=object)

In [5]:
my_first_df.string_column.nunique()

100

In [6]:
# тук проверяваме коя от колонките има само уникални стойности:
df_length = my_first_df.shape[0]
# премахваме колоната-списък, защото unique не работи там
for col in my_first_df.drop('list_column', axis=1).columns: 
   unique_values = my_first_df[col].nunique()
   if unique_values == df_length:
      print(f'Column {col} contains only unique values.')

Column string_column contains only unique values.
Column date_column contains only unique values.
Column dollar_values contains only unique values.


Често искаме да приложим разнообразни методи върху самите стойности в рамката данни, напр. да превърнем низовете в малки или главни букви, или пък да изберем определена част от низа.

In [7]:
# Превръщаме всички букви в главни. Вж. също методите capitalize и lower. 
# Удобни за стандартизиране на низови стойности.
my_first_df.string_column.str.upper().head()

0    STRING_VALUE_0
1    STRING_VALUE_1
2    STRING_VALUE_2
3    STRING_VALUE_3
4    STRING_VALUE_4
Name: string_column, dtype: object

In [8]:
#избираме първите 6 символа
my_first_df.string_column.str[:6].head()

0    string
1    string
2    string
3    string
4    string
Name: string_column, dtype: object

In [9]:
# Често методите върху низове са полезни ако искаме да извлечем числени стойности от тях. 
# Например, ако искаме да разделим стойностите в колонката dollar_values на 2,
# долната операция ще върне грешка:
# my_first_df.dollar_values / 2
# Първо трябва да променим стойностите на количествени:
dollar_values_only_numbers = my_first_df.dollar_values.str.replace('$', '', regex=False)
# Използваме метода replace, за да премахнем всичките символи за долар.
dollar_values_numerical = dollar_values_only_numbers.astype(int)
dollar_values_numerical.head() / 2

0      0.0
1     50.0
2    100.0
3    150.0
4    200.0
Name: dollar_values, dtype: float64

Често ни се налага да превърнем количествена променлива в качествена, напр. възрастови групи. Това става чрез метода pandas.cut().

In [10]:
n_categories = 5
dollar_categories = pd.cut(
    dollar_values_numerical,
    bins = n_categories
)
dollar_categories.value_counts()

(-9.9, 1980.0]      20
(1980.0, 3960.0]    20
(3960.0, 5940.0]    20
(5940.0, 7920.0]    20
(7920.0, 9900.0]    20
Name: dollar_values, dtype: int64

Нека зададем категориите върху рамката си данни и да извлечем извадка, за да проверим дали съответстват.

In [11]:
my_first_df['dollar_categories'] = dollar_categories
sample_check = my_first_df.sample(10)
sample_check

,string_column,binary_column,list_column,date_column,three_categories,dollar_values,dollar_categories
81,string_value_81,True,"[81, 162, 243]",01/01/1981,C,$8100,"(7920.0, 9900.0]"
83,string_value_83,True,"[83, 166, 249]",01/01/1983,C,$8300,"(7920.0, 9900.0]"
22,string_value_22,False,"[22, 44, 66]",01/01/1922,A,$2200,"(1980.0, 3960.0]"
53,string_value_53,True,"[53, 106, 159]",01/01/1953,B,$5300,"(3960.0, 5940.0]"
3,string_value_3,True,"[3, 6, 9]",01/01/1903,A,$300,"(-9.9, 1980.0]"
99,string_value_99,True,"[99, 198, 297]",01/01/1999,C,$9900,"(7920.0, 9900.0]"
39,string_value_39,True,"[39, 78, 117]",01/01/1939,B,$3900,"(1980.0, 3960.0]"
49,string_value_49,True,"[49, 98, 147]",01/01/1949,B,$4900,"(3960.0, 5940.0]"
58,string_value_58,False,"[58, 116, 174]",01/01/1958,B,$5800,"(3960.0, 5940.0]"
23,string_value_23,True,"[23, 46, 69]",01/01/1923,A,$2300,"(1980.0, 3960.0]"


Стойностите изглеждат правилни, нека им зададем подходящи етикети.

In [12]:
dollar_categories_labelled = pd.cut(
    dollar_values_numerical,
    bins = n_categories,
    labels = ['dollar_category_'+str(i) for i in range(1, n_categories+1)]
)
my_first_df['dollar_categories'] = dollar_categories_labelled
sample_check = my_first_df.sample(10)
sample_check

,string_column,binary_column,list_column,date_column,three_categories,dollar_values,dollar_categories
6,string_value_6,False,"[6, 12, 18]",01/01/1906,A,$600,dollar_category_1
50,string_value_50,False,"[50, 100, 150]",01/01/1950,B,$5000,dollar_category_3
19,string_value_19,True,"[19, 38, 57]",01/01/1919,A,$1900,dollar_category_1
16,string_value_16,False,"[16, 32, 48]",01/01/1916,A,$1600,dollar_category_1
82,string_value_82,False,"[82, 164, 246]",01/01/1982,C,$8200,dollar_category_5
59,string_value_59,True,"[59, 118, 177]",01/01/1959,B,$5900,dollar_category_3
65,string_value_65,True,"[65, 130, 195]",01/01/1965,B,$6500,dollar_category_4
72,string_value_72,False,"[72, 144, 216]",01/01/1972,C,$7200,dollar_category_4
79,string_value_79,True,"[79, 158, 237]",01/01/1979,C,$7900,dollar_category_4
14,string_value_14,False,"[14, 28, 42]",01/01/1914,A,$1400,dollar_category_1


Ако получим триизмерни данни, където всеки запис на колонката съдържа списък, често помага да го превърнем в отделни колонки.

In [13]:
my_first_df.list_column.head()

0     [0, 0, 0]
1     [1, 2, 3]
2     [2, 4, 6]
3     [3, 6, 9]
4    [4, 8, 12]
Name: list_column, dtype: object

In [14]:
element_2 = my_first_df.list_column.str[1]
element_2.head()

0    0
1    2
2    4
3    6
4    8
Name: list_column, dtype: int64

Когато работим с алгоритми, обикновено се налага да превърнем категорийните си стойности в числа. Най-простият начин да стане това е да зададем числена стойност на всяка категория чрез речник и да я приложим върху колонката.

In [15]:
my_first_df.three_categories.sample(5, random_state=42) 
# random_state подсигурява едни и същи псевдо-произволни стойности

83    C
53    B
70    C
45    B
44    B
Name: three_categories, dtype: object

In [16]:
value_recode = {
    'A': 1,
    'B': 2,
    'C': 3
}
three_categories_recoded = my_first_df.three_categories.map(value_recode)
three_categories_recoded.sample(5, random_state=42)

83    3
53    2
70    3
45    2
44    2
Name: three_categories, dtype: int64

Въпреки това, ако трите категории нямат ординално взаимоотношение (напр. имена на градове), това би подвело повечето алгоритми да считат, че такова съществува. Затова, ако работим с номинални качествени стойности, най-сигурно е да ги разбием на фиктивни променливи (dummy variables, а процесът е още познат като one-hot encoding), където всяка стойност бива превърната в отделна променлива, където тя получава 0 или 1 в зависимост от това дали е отбелязана.

In [17]:
one_hot_recode = pd.get_dummies(my_first_df.three_categories)
one_hot_recode.sample(5, random_state=42)

,A,B,C
83,0,0,1
53,0,1,0
70,0,0,1
45,0,1,0
44,0,1,0


### Задачи

 - Намерете и заредете набор данни от Kaggle. Идентифицирайте качествена променлива и я опишете (вкл. дескриптивна статистика).
 - Рекодирайте променливата във фиктивни променливи.